# Trying out different middleware integrations

## Summarization Middleware

Just trying out here what is seen from an `astream_events` perspective when summarization occurres in middleware:

In [2]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.messages import HumanMessage


agent = create_agent(
        model=ChatOpenAI(model="gpt-4o-mini"),
        tools=[],
        system_prompt="You are an helpful AI assistant that talks like the man with no name from The good, the bad and the ugly",
        middleware=[
            SummarizationMiddleware(
                model=ChatOpenAI(model="gpt-4o-mini"),
                max_tokens_before_summary=10,  # Trigger summarization at 20000 tokens
                messages_to_keep=1,  # Keep last 10 messages after summary
            ),
        ]
    )

In [4]:
async for event in agent.astream_events({"messages": [HumanMessage(content="Hello, how are you today?")]}):
    print(event)

{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Hello, how are you today?', additional_kwargs={}, response_metadata={})]}}, 'name': 'LangGraph', 'tags': [], 'run_id': '6879eb8d-3e91-4ea5-a08f-aea739e8f63b', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Hello, how are you today?', additional_kwargs={}, response_metadata={}, id='08d0d53c-1eea-4bd5-99be-4f198dfb6f57')]}}, 'name': 'SummarizationMiddleware.before_model', 'tags': ['graph:step:1'], 'run_id': 'a9f8a50d-7f14-4a0b-ae57-d0fc7a553a08', 'metadata': {'langgraph_step': 1, 'langgraph_node': 'SummarizationMiddleware.before_model', 'langgraph_triggers': ('branch:to:SummarizationMiddleware.before_model',), 'langgraph_path': ('__pregel_pull', 'SummarizationMiddleware.before_model'), 'langgraph_checkpoint_ns': 'SummarizationMiddleware.before_model:d95010ea-623a-b6a9-8966-6f3b94245eb9'}, 'parent_ids': ['6879eb8d-3e91-4ea5-a08f-aea739e8f63